In [1]:
# import
import os
import glob
import psycopg2
import datetime
import numpy as np
import pandas as pd

# set directory
dir = 'C:\\oasis\\git\\data\\microdata\\'

# set table name
table = 'm1.aggregate_bld'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [8]:
# 파일 불러와서 
for file in glob.glob(dir + "*집합.xlsx"):
    
    df = pd.read_excel(file)
    
    # 년도마다 ID 컬럼 유무가 달라서 사용할 컬럼 선택함
    df = df[['광역상권', '상권명', '소재지', '대지면적', '건축면적', '연면적', '건폐율', '용적률', '건물구조',
       '주용도', '지상층수', '지하층수', '사용승인_준공_일자', '승강기수', '주차장면적', '용도지역', '지하철역출구',
       '버스정류장', '주요교차로', '전환률', '전통시장', '기준층', '조사층', '임차인사용현황구분', '임차인업종대분류',
       '임차인업종소분류', '전용면적(호)', '공용면적(호)', '임차인별면적합계', '소유자수(호)', '소유자형태',
       '제1월계약임대료_보증금(총액)', '제1월계약임대료_월세(총액)', '제1월계약임대료_m당월세임대료',
       '제1월시장임대료_보증금(총액)', '제1월시장임대료_월세(총액)', '제1월시장임대료_m당월세임대료', '제1월관리비총액',
       '제1월관리비실비총액', '제1월공실', '제2월계약임대료_보증금(총액)', '제2월계약임대료_월세(총액)',
       '제2월계약임대료_m당월세임대료', '제2월시장임대료_보증금(총액)', '제2월시장임대료_월세(총액)',
       '제2월시장임대료_m당월세임대료', '제2월관리비총액', '제2월관리비실비총액', '제2월공실',
       '제3월계약임대료_보증금(총액)', '제3월계약임대료_월세(총액)', '제3월계약임대료_m당월세임대료',
       '제3월시장임대료_보증금(총액)', '제3월시장임대료_월세(총액)', '제3월시장임대료_m당월세임대료', '제3월관리비총액',
       '제3월관리비실비총액', '제3월공실', '현계약개시일', '현계약기간', '최초계약개시일', '최종조사산정액_m당단가']]
    
    
    # 컬럼명 변경
    df = df.rename(columns={
          '광역상권' : 'commercial_wide_area'
        , '상권명' : 'cname'
        , '소재지' : 'sido'
        , '대지면적' : 'plat_area'
        , '건축면적' : 'arch_area'
        , '연면적' : 'tot_area'
        , '건폐율' : 'bc_rat'
        , '용적률' : 'vl_rat'
        , '건물구조' : 'building_structure'
        , '주용도' : 'main_purps_nm'
        , '지상층수' : 'ground_floor'
        , '지하층수' : 'under_ground_floor'
        , '사용승인_준공_일자' : 'useapr_day'
        , '승강기수' : 'elevator_gb'
        , '주차장면적' : 'parking_area'
        , '용도지역' : 'land_use'
        , '지하철역출구' : 'subway_distance'
        , '버스정류장' : 'bus_distance'
        , '주요교차로' : 'main_crossroad'
        , '전환률' : 'conversion_rate'
        , '전통시장' : 'traditional_market_gb'
        , '기준층' : 'typical_floor'
        , '조사층' : 'invest_floor'
        , '임차인사용현황구분' : 'tenant_usage_status'
        , '임차인업종대분류' : 'business_type_large'
        , '임차인업종소분류' : 'business_type_small'
        , '전용면적(호)' : 'net_leasable_area'
        , '공용면적(호)' : 'common_area'
        , '임차인별면적합계' : 'sum_area'
        , '소유자수(호)' : 'owner_ho'
        , '소유자형태' : 'owner_type'
        , '제1월계약임대료_보증금(총액)' : 'contract_deposit_1st'
        , '제1월계약임대료_월세(총액)' : 'contract_monthly_rent_sum_1st'
        , '제1월계약임대료_m당월세임대료' : 'contract_monthly_rent_1st'
        , '제1월시장임대료_보증금(총액)' : 'selling_deposit_1st'
        , '제1월시장임대료_월세(총액)' : 'selling_monthly_rent_sum_1st'
        , '제1월시장임대료_m당월세임대료' : 'selling_monthly_rent_1st'
        , '제1월관리비총액' : 'admin_cost_1st'
        , '제1월관리비실비총액' : 'admin_actual_cost_1st'
        , '제1월공실' : 'vacant_1'
        , '제2월계약임대료_보증금(총액)' : 'contract_deposit_2nd'
        , '제2월계약임대료_월세(총액)' : 'contract_monthly_rent_sum_2nd'
        , '제2월계약임대료_m당월세임대료' : 'contract_monthly_rent_2nd'
        , '제2월시장임대료_보증금(총액)' : 'selling_deposit_2nd'
        , '제2월시장임대료_월세(총액)' : 'selling_monthly_rent_sum_2nd'
        , '제2월시장임대료_m당월세임대료' : 'selling_monthly_rent_2nd'
        , '제2월관리비총액' : 'admin_cost_2nd'
        , '제2월관리비실비총액' : 'admin_actual_cost_2nd'
        , '제2월공실' : 'vacant_2'
        , '제3월계약임대료_보증금(총액)' : 'contract_deposit_3rd'
        , '제3월계약임대료_월세(총액)' : 'contract_monthly_rent_sum_3rd'
        , '제3월계약임대료_m당월세임대료' : 'contract_monthly_rent_3rd'
        , '제3월시장임대료_보증금(총액)' : 'selling_deposit_3rd'
        , '제3월시장임대료_월세(총액)' : 'selling_monthly_rent_sum_3rd'
        , '제3월시장임대료_m당월세임대료' : 'selling_monthly_rent_3rd'
        , '제3월관리비총액' : 'admin_cost_3rd'
        , '제3월관리비실비총액' : 'admin_actual_cost_3rd'
        , '제3월공실' : 'vacant_3'
        , '현계약개시일' : 'current_contract_date'
        , '현계약기간' : 'current_contract_term'
        , '최초계약개시일' : 'first_current_contract_date'
        , '최종조사산정액_m당단가' : 'invest_calculation_amt'
    })

    # base_ym 추가 
    df['base_ym'] = datetime.datetime.now().strftime("%Y%m")
    
    # 특수문자, nan 처리
    df = df.replace("%","", regex=True)
    df = df.replace({np.nan: None})

    # values 설정
    val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

    # insert
    for j in range(0, len(df), 10000):
        # DB Connect
        conn = psycopg2.connect(
            host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
            port=5439,
            dbname='dev',
            user='awsuser',
            password='cremaoAdmin1234qwer!!'
        )
        # 10000개 단위로 나누기
        tmp = df[j:j + 10000]

        # insert
        execute_mogrify(conn, tmp, table, val)
        print(datetime.datetime.now(), ' : ', j)
        tmp = pd.DataFrame()

    # row count
    print(f'{len(df)} Done : {file}')

2022-12-16 10:13:47.318190  :  0
2022-12-16 10:14:00.321306  :  10000
2022-12-16 10:14:05.824438  :  20000
22963 Done : C:\oasis\git\data\microdata\2020_중대형상가.xlsx
2022-12-16 10:14:35.595473  :  0
2022-12-16 10:14:48.042780  :  10000
2022-12-16 10:14:56.155912  :  20000
26410 Done : C:\oasis\git\data\microdata\2021_중대형상가.xlsx


### 짤막

In [1]:
import psycopg2
import datetime

import numpy  as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [10]:
df = pd.read_csv('./집합2021.csv')

df = df.replace("%","", regex=True)
df = df.replace({np.nan: None})

In [ ]:
df.columns = ['commercial_wide_area', 'cname', 'sido', 'plat_area', 'arch_area','tot_area', 'bc_rat', 'vl_rat', 'building_structure', 'main_purps_nm',
       'ground_floor', 'under_ground_floor', 'useapr_day', 'elevator_gb','parking_area', 'land_use', 'subway_distance', 'bus_distance',
       'main_crossroad', 'conversion_rate', 'traditional_market_gb', 'typical_floor', 'invest_floor', 'tenant_usage_status',
       'business_type_large', 'business_type_small', 'net_leasable_area','common_area', 'sum_area', 'owner_ho', 'owner_type',
       'contract_deposit_1st', 'contract_monthly_rent_sum_1st','contract_monthly_rent_1st', 'selling_deposit_1st',
       'selling_monthly_rent_sum_1st', 'selling_monthly_rent_1st','admin_cost_1st', 'admin_actual_cost_1st', 'vacant_1',
       'contract_deposit_2nd', 'contract_monthly_rent_sum_2nd','contract_monthly_rent_2nd', 'selling_deposit_2nd',
       'selling_monthly_rent_sum_2nd', 'selling_monthly_rent_2nd','admin_cost_2nd', 'admin_actual_cost_2nd', 'vacant_2',
       'contract_deposit_3rd', 'contract_monthly_rent_sum_3rd','contract_monthly_rent_3rd', 'selling_deposit_3rd',
       'selling_monthly_rent_sum_3rd', 'selling_monthly_rent_3rd','admin_cost_3rd', 'admin_actual_cost_3rd', 'vacant_3',
       'current_contract_date', 'current_contract_term','first_current_contract_date', 'invest_calculation_amt']

In [8]:
# DW INSERT
import psycopg2

table = 'm1.aggregate_bld'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))    
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        # conn.rollback()
        conn.commit()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):
    
    # DB Connect
    while True:
        try:
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )
            break
        except Exception as err:
            pass

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    tmp = pd.DataFrame()

print('Done, ', len(df))

Done,  0
Done,  0
Done,  0
